In [ ]:

import warnings

import matplotlib.pyplot as plt
# import MultiTrade
import numpy as np
import pandas as pd
from IPython.display import display
from neuralforecast.core import NeuralForecast

from neuralforecast.auto import AutoBiTCN, AutoNBEATS, AutoTFT
from neuralforecast.losses.pytorch import RMSE

# from statsforecast import StatsForecast

from utils.utils import build_market_image,train_test_split_data
from configs import defaults
import datetime
warnings.filterwarnings("ignore")

In [2]:
COIN_PAIRS=defaults.COIN_PAIRS
target_pair=defaults.target_pair
time_frame=defaults.time_frame
DATA_DIR=defaults.DATA_DIR

In [3]:

horizon = 7
context_length=5*horizon
split_n_days=30
round_time='H'
id_col='symbol'
save_path=defaults.forecasting_model_path

In [4]:

data=build_market_image(target_pair=target_pair,time_frame=time_frame,axis=0,data_dir=DATA_DIR,only_target=True)
# data.index

train_data,test_data=train_test_split_data(data=data,n_days=30)

# data['symbol'].unique()

# test_data
data

['data/binanceus-DOGEUSDT-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 11330.46it/s]
1it [00:00,  7.19it/s]

2024-10-20 13:00:00 2024-11-19 13:00:00


,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K,ds,unique_id,symbol
date_close,,,,,,,,,,,,,,,,,,,,
2024-01-01 10:00:00,0.08975,0.08976,0.08975,0.08976,6209.0,0.08976,-0.065217,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-01-01 10:00:00,25,DOGEUSDT
2024-01-01 11:00:00,0.08977,0.09012,0.08972,0.09012,13750.0,0.09012,-0.021739,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000008,0.000004,100.000000,0.000000,2024-01-01 11:00:00,25,DOGEUSDT
2024-01-01 12:00:00,0.09008,0.09008,0.08992,0.08992,583.0,0.08992,0.021739,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000004,0.000004,62.566845,0.000000,2024-01-01 12:00:00,25,DOGEUSDT
2024-01-01 13:00:00,0.08995,0.09046,0.08995,0.09040,12750.0,0.09040,0.065217,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000020,0.000009,80.974284,0.000000,2024-01-01 13:00:00,25,DOGEUSDT
2024-01-01 14:00:00,0.09018,0.09053,0.09018,0.09053,5594.0,0.09053,0.108696,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000033,0.000016,83.360761,0.000000,2024-01-01 14:00:00,25,DOGEUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-17 15:00:00,0.36347,0.36425,0.35770,0.36143,223141.0,0.36143,0.152174,0.5,0.033333,0.379452,0.374743,0.361372,0.348002,-0.001727,-0.002703,47.544678,60.921502,2024-11-17 15:00:00,25,DOGEUSDT
2024-11-17 16:00:00,0.36137,0.36550,0.35834,0.36372,137563.0,0.36372,0.195652,0.5,0.033333,0.379452,0.373451,0.360964,0.348477,-0.001471,-0.002457,49.746258,67.434585,2024-11-17 16:00:00,25,DOGEUSDT
2024-11-17 17:00:00,0.36445,0.36762,0.36062,0.36762,196547.0,0.36762,0.239130,0.5,0.033333,0.379452,0.373621,0.361020,0.348419,-0.000942,-0.002154,53.338130,74.415452,2024-11-17 17:00:00,25,DOGEUSDT


In [5]:
front=['y','ds','symbol']
exo_gen_cols=data.filter(like='feature_').columns.tolist()
cols=front+[c for c in exo_gen_cols if c not in front]

train_data=train_data[cols]
test_data=test_data[cols]
# train_data
# data[id_col].unique()
# 

In [6]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [7]:
forecast_horizon=6
backend='optuna'
BiTCN_config=AutoBiTCN.get_default_config(h=forecast_horizon,backend=backend)

TFT_config=AutoTFT.get_default_config(h=forecast_horizon,backend=backend)
NBEATS_config=AutoNBEATS.get_default_config(h=forecast_horizon,backend=backend)

In [8]:
BiTCN_MODEL= AutoBiTCN(h=forecast_horizon,
                  loss=RMSE(),
                  config=BiTCN_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

TFT_MODEL= AutoTFT(h=forecast_horizon,
                  loss=RMSE(),
                  config=TFT_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

NBEATS_MODEL= AutoNBEATS(h=forecast_horizon,
                  loss=RMSE(),
                  config=NBEATS_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

models=[BiTCN_MODEL,TFT_MODEL,NBEATS_MODEL]

In [9]:

model = NeuralForecast(models=models,
                    local_scaler_type='robust',
                    
                     freq='1h')

In [10]:
model.fit(train_data,
        val_size=horizon,
        time_col='ds',                    
        target_col='y',
        id_col=id_col,
        
        )


Seed set to 7
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | RMSE          | 0      | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | lin_hist      | Linear        | 64     | train
4 | drop_hist     | Dropout       | 0      | train
5 | net_bwd       | Sequential    | 21.0 K | train
6 | drop_temporal | Dropout       | 0      | train
7 | temporal_lin1 | Linear        | 608    | train
8 | temporal_lin2 | Linear        | 198    | train
9 | output_lin    | Linear        | 33     | train
--------------------------------------------------------
21.9 K    Trainable params
0         Non-trainable params
21.9 K    Total params
0.087     Total estimated model params size (MB)
30        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [16]:
model.save(save_path,overwrite=True)
save_path

'DOGEUSDTForecastingModel/'

In [12]:
test_dates=test_data.sort_values('ds')['ds'].unique()
last_date=train_data['ds'].unique()[-1]

In [13]:
test_data

,y,ds,symbol,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K
0,0.13779,2024-10-20 13:00:00,DOGEUSDT,0.065217,0.5,0.133333,0.302740,0.144459,0.141576,0.138693,-0.000004,0.000345,38.159728,0.000000
1,0.13922,2024-10-20 14:00:00,DOGEUSDT,0.108696,0.5,0.133333,0.302740,0.144532,0.141533,0.138535,-0.000193,0.000237,43.262934,23.275862
2,0.13905,2024-10-20 15:00:00,DOGEUSDT,0.152174,0.5,0.133333,0.302740,0.144605,0.141494,0.138384,-0.000352,0.000119,42.810640,27.741935
3,0.13857,2024-10-20 16:00:00,DOGEUSDT,0.195652,0.5,0.133333,0.302740,0.144737,0.141423,0.138109,-0.000511,-0.000007,41.491646,20.000000
4,0.14008,2024-10-20 17:00:00,DOGEUSDT,0.239130,0.5,0.133333,0.302740,0.144745,0.141389,0.138032,-0.000510,-0.000107,47.021470,44.354839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,0.36143,2024-11-17 15:00:00,DOGEUSDT,0.152174,0.5,0.033333,0.379452,0.374743,0.361372,0.348002,-0.001727,-0.002703,47.544678,60.921502
675,0.36372,2024-11-17 16:00:00,DOGEUSDT,0.195652,0.5,0.033333,0.379452,0.373451,0.360964,0.348477,-0.001471,-0.002457,49.746258,67.434585
676,0.36762,2024-11-17 17:00:00,DOGEUSDT,0.239130,0.5,0.033333,0.379452,0.373621,0.361020,0.348419,-0.000942,-0.002154,53.338130,74.415452
677,0.36722,2024-11-17 18:00:00,DOGEUSDT,0.282609,0.5,0.033333,0.379452,0.373282,0.360922,0.348561,-0.000549,-0.001833,52.920342,71.454219


In [14]:
fcst_df = model.predict()
plot_df=train_data.groupby('symbol').tail(context_length)
plot_df
# fcst_df.tail()

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

,y,ds,symbol,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K
7001,0.14529,2024-10-19 03:00:00,DOGEUSDT,-0.369565,0.333333,0.100000,0.30000,0.143898,0.137359,0.130820,0.003088,0.002703,76.546688,93.529412
7002,0.14422,2024-10-19 04:00:00,DOGEUSDT,-0.326087,0.333333,0.100000,0.30000,0.144923,0.137834,0.130745,0.003247,0.002812,72.104398,84.375000
7003,0.14320,2024-10-19 05:00:00,DOGEUSDT,-0.282609,0.333333,0.100000,0.30000,0.145575,0.138256,0.130937,0.003254,0.002901,68.050147,72.268908
7004,0.14412,2024-10-19 06:00:00,DOGEUSDT,-0.239130,0.333333,0.100000,0.30000,0.146302,0.138718,0.131134,0.003296,0.002980,69.704757,79.622266
7005,0.14515,2024-10-19 07:00:00,DOGEUSDT,-0.195652,0.333333,0.100000,0.30000,0.147127,0.139214,0.131302,0.003373,0.003058,71.485212,89.860835
7006,0.14383,2024-10-19 08:00:00,DOGEUSDT,-0.152174,0.333333,0.100000,0.30000,0.147411,0.139734,0.132057,0.003290,0.003105,66.122011,76.739563
7007,0.14473,2024-10-19 09:00:00,DOGEUSDT,-0.108696,0.333333,0.100000,0.30000,0.147677,0.140300,0.132923,0.003259,0.003135,67.890857,85.685885
7008,0.14459,2024-10-19 10:00:00,DOGEUSDT,-0.065217,0.333333,0.100000,0.30000,0.147880,0.140807,0.133733,0.003186,0.003146,67.302187,84.294235
7009,0.14623,2024-10-19 11:00:00,DOGEUSDT,-0.021739,0.333333,0.100000,0.30000,0.148351,0.141342,0.134334,0.003224,0.003161,70.526201,97.207859
7010,0.14239,2024-10-19 12:00:00,DOGEUSDT,0.021739,0.333333,0.100000,0.30000,0.148376,0.141582,0.134788,0.002911,0.003111,56.482950,56.415695


In [15]:
from statsforecast import StatsForecast


ModuleNotFoundError: No module named 'statsforecast'

In [ ]:
StatsForecast.plot(plot_df, fcst_df, engine='matplotlib',id_col=id_col, max_insample_length=context_length,)


In [ ]:
pred_frame=test_data.copy()

In [ ]:
model=NeuralForecast.load(save_path)

In [ ]:
fcst_df = model.predict(pred_frame)

# fcst_df.head()

In [ ]:
StatsForecast.plot(pred_frame, fcst_df, engine='matplotlib',id_col=id_col, max_insample_length=context_length,)


In [ ]:
test_data

In [ ]:
# test_data=test_data.drop('close_date',axis=1)
test_data[exo_gen_cols].values.dtype

In [ ]:
model.dataset, model.uids, model.last_dates, model.ds = model._prepare_fit(
                df=test_data,
                static_df=None,
                sort_df=model.sort_df,
                predict_only=False,
                id_col=model.id_col,
                time_col=model.time_col,
                target_col=model.target_col,
            )


In [ ]:
test_pred_insample = model.predict_insample(step_size=horizon)


In [ ]:
test_pred_insample['mean']=test_pred_insample.filter(like='Auto').mean(axis=1)
# test_pred_insample['mean']
# test_pred_insample['mean']
# test_pred_insample.filter(like='Auto').mean(axis=1)
# test_pred_insample.filter(like='Auto').mean(axis=1)

In [ ]:
# test_pred_insample.columns=test_pred_insample.columns.str.replace('Auto','')

In [ ]:
test_data['close']=test_data['y'].copy()


In [ ]:
idx,cut_data=next(iter(test_pred_insample.groupby(['cutoff','symbol'])))
def flatten_preds(idx,cut_data):
    t_off_pred,symb=idx
    pred_cols=cut_data.filter(like='Auto').columns
    cut_data['mean_pred']=cut_data[pred_cols].mean(axis=1)
    cut_data=cut_data.drop(pred_cols,axis=1)
    t_cut=cut_data.T

    t_cut=t_cut.drop('cutoff')
    t_cut=t_cut.drop('ds')
    t_cut=t_cut.drop('y')
    # print(t_cut)
    h_cols=[f'H{i}' for i in range(horizon-1)]
    # print (h_cols)
    t_cut.columns=h_cols


    flat_cols=[f'feature_{model}_{horizon}' for model in t_cut.index for horizon in t_cut.columns]
    pred_values=t_cut.values.flatten()

    flat_df=pd.DataFrame([pred_values],columns=flat_cols)
    id_cols=['ds','symbol']

    flat_df[id_cols]=t_off_pred,symb
    flat_df=  flat_df[id_cols+flat_cols]
    return flat_df

def simulate_forecasts(model,df):
    model.dataset, model.uids, model.last_dates, model.ds = model._prepare_fit(
                df=test_data,
                static_df=None,
                sort_df=model.sort_df,
                predict_only=True,
                id_col=model.id_col,
                time_col=model.time_col,
                target_col=model.target_col,
            )
    
    
    return model.predict_insample(step_size=1)

def prepare_forecast_data(model,test_data):
    
    pred_df= simulate_forecasts(model,test_data)
    flattened_preds=pd.concat([flatten_preds(idx,cut_data) for idx,cut_data in pred_df.groupby(['cutoff','symbol'])])
    return flattened_preds



prepare_forecast_data(model,test_data)

In [ ]:
cut_data
for symb,cut in test_pred_insample.groupby('symbol'):
    cut['mean']=cut.filter(like='Auto').mean(axis=1)

    plt.figure(figsize=(10, 5))
    
    plt.plot(cut['ds'], cut['y'], label='True')
    plt.plot(cut['ds'], cut['mean'] ,label='mean Forecast')
    for model in cut.filter(like='Auto').columns:
        plt.plot(cut['ds'], cut[model], label=f'{model} Forecast')
    # plt.plot(cut['ds'], cut['NBEATS'], label='NBEATS Forecast')
    # plt.plot(cut['ds'], cut['BiTCN'], label='BiTCN Forecast')
    # plt.plot(cut['ds'], cut['TFT'], label='TFT Forecast')
    # plt.axvline(cut['ds'].iloc[-12], color='black', linestyle='--', label='Train-Test Split')
    plt.xlabel('Timestamp [t]')
    plt.ylabel(f'{symb} Price')
    plt.grid()
    plt.legend()

In [ ]:
# StatsForecast.plot(test_pred_insample, engine='matplotlib',time_col='ds',id_col=id_col, max_insample_length=horizon,)
# StatsForecast.plot(test_pred_insample, engine='matplotlib',time_col='ds',id_col=id_col,target_col='AutoNBEATS', max_insample_length=horizon,)
